In [1]:
from pubg_util import mysql, load_schema, notifier
from sphynx import sphynx, NODE_SMALL, NODE_MEDIUM, NODE_LARGE
from pyspark.sql.functions import *
import pandas as pd

spark = sphynx.get_spark(executor_count=8, app_name='pdu_syp2', node_spec=NODE_LARGE)

Spark cluster not assigned. creating a new one...
Node spec: 8 executors with 48G RAM each
Job Port 4049 is assigned for requested cluster
Waiting for Spark master to be available...
Spark master launched!
Creating new Spark session, name: pdu_syp2...
Waiting for all executors ready...
All executors connected!
Complete! elapsed time: 00:00:25


In [58]:
sphynx.stop()

In [2]:
items = mysql.read_table(spark, "metainfo", "item_meta_sales")
wsus_product_id = items.where(lower(col("product_name")).like("%progressive%")).select(col("product_id")).distinct().toPandas()["product_id"].tolist()
weapon_product_list = items.where(col("sub_category") == "weapon").select(col("product_id")).distinct().toPandas().product_id.tolist()

In [3]:
weapon_name = items.where(col("sub_category").isin(["weapon", "crate"])).select("product_id", "product_name", "part").distinct().toPandas()

In [4]:
weapon_name.head()

,product_id,product_name,part
0,itemdesc.13000250,ROSEBOUND WEAPON BUNDLE,crate
1,itemdesc.12012003,[PROGRESSIVE] Trick-or-treat - M416,m416
2,itemdesc.13000377,COMMANDO NECK SLEEVE,crate
3,itemdesc.13000368,PURE HOODIE,crate
4,itemdesc.14200002,BLUE COUPON,crate


In [3]:
# 6월 성장형 무기 스킨 출시 후 2주간 contraband로 아이템 획득

device = 'pc' 
env = 'live' 
log_name = 'ItemUpgraded'
start_date = '2021-06-02'
end_date = '2021-06-15' 
june_item_upgrade = load_schema.lobby(spark, device, env, log_name, start_date, end_date)

In [12]:
# 8월 성장형 무기 스킨 출시 후 2주간 contraband로 아이템 획득

device = 'pc' 
env = 'live' 
log_name = 'ItemUpgraded'
start_date = '2021-08-11'
end_date = '2021-08-24' 
aug_item_upgrade = load_schema.lobby(spark, device, env, log_name, start_date, end_date)

In [18]:
# 10월 성장형 무기 스킨 출시 후 2주간 contraband로 아이템 획득

device = 'pc' 
env = 'live' 
log_name = 'ItemUpgraded'
start_date = '2021-10-06'
end_date = '2021-10-19' 
oct_item_upgrade = load_schema.lobby(spark, device, env, log_name, start_date, end_date)

In [5]:
# 6월 성장형 무기 스킨 출시 후 4주간 아이템 착용

device = 'pc' 
env = 'live' 
log_name = 'ItemEquipped'
start_date = '2021-06-02'
end_date = '2021-06-29' 
june_item_equipped = load_schema.lobby(spark, device, env, log_name, start_date, end_date)

In [13]:
# 8월 성장형 무기 스킨 출시 후 4주간 아이템 착용

device = 'pc' 
env = 'live' 
log_name = 'ItemEquipped'
start_date = '2021-08-11'
end_date = '2021-09-07' 
aug_item_equipped = load_schema.lobby(spark, device, env, log_name, start_date, end_date)

In [19]:
# 10월 성장형 무기 스킨 출시 후 4주간 아이템 착용

device = 'pc' 
env = 'live' 
log_name = 'ItemEquipped'
start_date = '2021-10-06'
end_date = '2021-11-02' 
oct_item_equipped = load_schema.lobby(spark, device, env, log_name, start_date, end_date)

In [9]:
june_item_equipped.printSchema()

root
 |-- date: string (nullable = true)
 |-- AccountId: string (nullable = true)
 |-- ItemId: string (nullable = true)
 |-- PartId: string (nullable = true)
 |-- PlatformProvider: string (nullable = true)
 |-- Provider: string (nullable = true)
 |-- Time: string (nullable = true)
 |-- Type: string (nullable = true)



In [6]:
june_wsus_user = june_item_upgrade.where(col("ItemDescId") == "itemdesc.12012001").select("AccountId").distinct().toPandas()["AccountId"].tolist()

In [14]:
aug_wsus_user = aug_item_upgrade.where(col("ItemDescId") == "itemdesc.12012002").select("AccountId").distinct().toPandas()["AccountId"].tolist()

In [20]:
oct_wsus_user = oct_item_upgrade.where(col("ItemDescId") == "itemdesc.12012003").select("AccountId").distinct().toPandas()["AccountId"].tolist()

In [7]:
import pickle
with open("./data/june_wsus_user.pickle", "wb") as f:
    pickle.dump(june_wsus_user, f)

In [15]:
import pickle
with open("./data/aug_wsus_user.pickle", "wb") as f:
    pickle.dump(aug_wsus_user, f)

In [21]:
import pickle
with open("./data/oct_wsus_user.pickle", "wb") as f:
    pickle.dump(oct_wsus_user, f)

In [12]:
june_item_equipped_df2 = pd.read_csv("./data/june_item_equipped_df.csv")

In [17]:
part_list = june_item_equipped_df2.PartId.unique().tolist()
with open("./data/partId_list.pickle", "wb") as f:
    pickle.dump(part_list, f)

In [10]:
june_item_equipped_df = june_item_equipped.where((col("AccountId").isin(june_wsus_user)) & (col("ItemId").isin(weapon_product_list))).toPandas()

In [11]:
june_item_equipped_df.to_csv("./data/june_item_equipped_df.csv", index=False)

weapon이 아닌 crate도 넣었을 때 

In [19]:
import pickle
with open("./data/june_wsus_user.pickle", "rb") as f:
    june_wsus_user = pickle.load(f)
june_item_equipped_df = june_item_equipped.where((col("AccountId").isin(june_wsus_user)) & (col("PartId").isin(part_list))).toPandas()

----------------------------------------
Exception happened during processing of request from ('127.0.0.1', 58686)
Traceback (most recent call last):
  File "/opt/conda/lib/python3.6/socketserver.py", line 317, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/opt/conda/lib/python3.6/socketserver.py", line 348, in process_request
    self.finish_request(request, client_address)
  File "/opt/conda/lib/python3.6/socketserver.py", line 361, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/opt/conda/lib/python3.6/socketserver.py", line 721, in __init__
    self.handle()
  File "/usr/local/spark/python/pyspark/accumulators.py", line 269, in handle
    poll(accum_updates)
  File "/usr/local/spark/python/pyspark/accumulators.py", line 241, in poll
    if func():
  File "/usr/local/spark/python/pyspark/accumulators.py", line 245, in accum_updates
    num_updates = read_int(self.rfile)
  File "/usr/local/spark/python/pysp

In [9]:
june_item_equipped_df.to_csv("./data/june_item_equipped_df2.csv", index=False)

In [16]:
aug_item_equipped_df = aug_item_equipped.where((col("AccountId").isin(aug_wsus_user)) & (col("ItemId").isin(weapon_product_list))).toPandas()

In [17]:
aug_item_equipped_df.to_csv("./data/aug_item_equipped_df.csv", index=False)

In [22]:
oct_item_equipped_df = oct_item_equipped.where((col("AccountId").isin(oct_wsus_user)) & (col("ItemId").isin(weapon_product_list))).toPandas()

In [23]:
oct_item_equipped_df.to_csv("./data/oct_item_equipped_df.csv", index=False)

In [28]:
june_wsus_equip_user = june_item_equipped_df[june_item_equipped_df.ItemId == "itemdesc.12012001"].AccountId.unique().tolist()

In [35]:
aug_wsus_equip_user = aug_item_equipped_df[aug_item_equipped_df.ItemId == "itemdesc.12012002"].AccountId.unique().tolist()

In [33]:
oct_wsus_equip_user = oct_item_equipped_df[oct_item_equipped_df.ItemId == "itemdesc.12012003"].AccountId.unique().tolist()

In [44]:
june_item_equipped_df = pd.merge(june_item_equipped_df, weapon_name, left_on="ItemId", right_on="product_id")
aug_item_equipped_df = pd.merge(aug_item_equipped_df, weapon_name, left_on="ItemId", right_on="product_id")
oct_item_equipped_df = pd.merge(oct_item_equipped_df, weapon_name, left_on="ItemId", right_on="product_id")

### 사용 무기

In [48]:
june_item_equipped_df[(june_item_equipped_df.AccountId.isin(june_wsus_equip_user))].groupby(["AccountId", "part", "date"], as_index=False)["Time"].count().sort_values(["AccountId", "date"]).to_csv("./data/june_equip.csv", index=False)

In [49]:
aug_item_equipped_df[(aug_item_equipped_df.AccountId.isin(aug_wsus_equip_user))].groupby(["AccountId", "part", "date"], as_index=False)["Time"].count().sort_values(["AccountId", "date"]).to_csv("./data/aug_equip.csv", index=False)

In [50]:
oct_item_equipped_df[(oct_item_equipped_df.AccountId.isin(oct_wsus_equip_user))].groupby(["AccountId", "part", "date"], as_index=False)["Time"].count().sort_values(["AccountId", "date"]).to_csv("./data/oct_equip.csv", index=False)

### 사용 스킨

In [51]:
june_item_equipped_df[(june_item_equipped_df.AccountId.isin(june_wsus_equip_user))].groupby(["AccountId", "product_name", "date"], as_index=False)["Time"].count().sort_values(["AccountId", "date"]).to_csv("./data/june_equip_product.csv", index=False)

In [52]:
aug_item_equipped_df[(aug_item_equipped_df.AccountId.isin(aug_wsus_equip_user))].groupby(["AccountId", "product_name", "date"], as_index=False)["Time"].count().sort_values(["AccountId", "date"]).to_csv("./data/aug_equip_product.csv", index=False)

In [53]:
oct_item_equipped_df[(oct_item_equipped_df.AccountId.isin(oct_wsus_equip_user))].groupby(["AccountId", "product_name", "date"], as_index=False)["Time"].count().sort_values(["AccountId", "date"]).to_csv("./data/oct_equip_product.csv", index=False)

----------------------------------------
Exception happened during processing of request from ('127.0.0.1', 38862)
Traceback (most recent call last):
  File "/opt/conda/lib/python3.6/socketserver.py", line 317, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/opt/conda/lib/python3.6/socketserver.py", line 348, in process_request
    self.finish_request(request, client_address)
  File "/opt/conda/lib/python3.6/socketserver.py", line 361, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/opt/conda/lib/python3.6/socketserver.py", line 721, in __init__
    self.handle()
  File "/usr/local/spark/python/pyspark/accumulators.py", line 269, in handle
    poll(accum_updates)
  File "/usr/local/spark/python/pyspark/accumulators.py", line 241, in poll
    if func():
  File "/usr/local/spark/python/pyspark/accumulators.py", line 245, in accum_updates
    num_updates = read_int(self.rfile)
  File "/usr/local/spark/python/pysp